In [1]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import gzip
from datetime import datetime, timedelta
from statistics import mean, median
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow
import tensorflow.keras as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, ReLU, LSTM, Conv1D, Conv2D, Embedding, Reshape
from tensorflow.keras.activations import sigmoid, tanh
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm
import csv
import random
import math

from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.metrics import f1_score as f1

import pickle

In [2]:
def extract_model(filename):
    model = tf.models.load_model(filename)
    return model
    
def retrieve_data(filename):
    if "combined" in filename:
        df = pd.read_csv(filename)
    else:
        df = pd.read_csv(filename, index_col=0)
    df["Date"] = pd.to_datetime(df["Date"])
    return df

def create_classification_data(df, lookback, column):
    rows = []
    columns = ['Date', column] # Date and SP500_relative_change_perc_1 from t-0 are added first as target variables 
    # create column names based on original with the addition of t-i where i is lookback
    for i in range(1, lookback + 1): # starts at 1 since we do not want t-0 variables apart from 'Date' and 'SP500_relative_change_perc_1'
        new_columns = df.columns.tolist() # starts at 1 to exclude 'Date' column
        for x in range(len(new_columns)):
            new_columns[x] = new_columns[x] + "_t-" + str(i)
        columns = columns + new_columns
    
    # create lookback data
    for i, row in enumerate(df.iterrows()):
        if i > lookback: # lookback cannot be determined for earlier rows
            new_row = [row[1]['Date'], row[1][column]]
#             new_row = [row[1][0], row[1][1]] # add target 'Date' and 'SP500_relative_change_perc_1 '
            for x in range(1, lookback + 1): # starts at 1 since we do not want t-0 variables apart from 'Date' and 'SP500_relative_change_perc_1'
                add_row = df.iloc[i - x].tolist() # starts at 1 to exclude 'Date' column
                new_row = new_row + add_row
            rows.append(new_row)
    
    df2 = pd.DataFrame(rows)
    df2.columns = columns
                       
    for col in columns:
        if col[:4] == "Date" and col != "Date":
            df2 = df2.drop([col], axis=1)
    return df2

def create_train(df, year_val, year_test, column, perc_train=None):
    # assumes years_train < year_val < year_test
    df["Date"] = pd.to_datetime(df["Date"])

    val = df[df['Date'].dt.year == year_val]
    test = df[df['Date'].dt.year == year_test]
    train = df[df['Date'].dt.year < year_val]
#     y_train = train[column]
    x_train = train
    return x_train


def create_val(df, year_val, year_test, column, perc_train=None):
    # assumes years_train < year_val < year_test
    df["Date"] = pd.to_datetime(df["Date"])
    val = df[df['Date'].dt.year == year_val]
    test = df[df['Date'].dt.year == year_test]
    train = df[df['Date'].dt.year < year_val]
#     y_val = val[column]
    x_val = val
#     display(x_val)
    return x_val

def create_test(df, year_val, year_test, column, perc_train=None):
    print("test", len(df))
    # assumes years_train < year_val < year_test
    df["Date"] = pd.to_datetime(df["Date"])

    val = df[df['Date'].dt.year == year_val]
    test = df[df['Date'].dt.year == year_test]
    train = df[df['Date'].dt.year < year_val]
#     y_test = test[column]
    x_test = test
    return x_test

def create_full(df, year_val, year_test, column, perc_train=None):
    df["Date"] = pd.to_datetime(df["Date"])
#     y = df[column]
    x = df
    
    return x

In [3]:
"""
    Assumptions:
        You can invest precisely at market open until (exactly) market close
        Investment amounts can be up to 10 decimal points
        Geen commission
        Geen exchange rates
        Trader can go long or short

    Requirements:
        Make sure to consider lookback of each model
        Add recent accuracy (yesterday (0 or 100%), last week, last 2 weeks) as additional features for final NN
"""


'\n    Assumptions:\n        You can invest precisely at market open until (exactly) market close\n        Investment amounts can be up to 10 decimal points\n        Geen commission\n        Geen exchange rates\n        Trader can go long or short\n\n    Requirements:\n        Make sure to consider lookback of each model\n        Add recent accuracy (yesterday (0 or 100%), last week, last 2 weeks) as additional features for final NN\n'

In [4]:
def predict_init(data, model, column):
    x = data.copy()
    x = x.drop(['Date'], axis=1)
    x = x.drop([column], axis=1)
    x = np.asarray(x)
    x = x.reshape((x.shape[0], 1, x.shape[1]))
    pred = model.predict(x)
    pred_new = []
    for i in pred:
        pred_new.append(i[0][0])
    
    data['pred'] = pred_new
    return data

def extract_data(instruments, data, mode):
    year_val = 2018
    year_test = 2019
    for symbol in instruments:
        column = symbol + "_relative_change_perc_1"
        ndir = symbol + "_dir"
        nmag = symbol + "_mag"

        data_dir = data[ndir]
        data_mag = data[nmag]
        if mode == "train":
            data[ndir] = create_train(data_dir, year_val, year_test, column)
            data[nmag] = create_train(data_mag, year_val, year_test, column)
        if mode == "val":
            data[ndir] = create_val(data_dir, year_val, year_test, column)
            data[nmag] = create_val(data_mag, year_val, year_test, column)
        if mode == "test":
            data[ndir] = create_test(data_dir, year_val, year_test, column)
            data[nmag] = create_test(data_mag, year_val, year_test, column)
        if mode == "full":
            data[ndir] = create_full(data_dir, year_val, year_test, column)
            data[nmag] = create_full(data_mag, year_val, year_test, column)
    return data

def init_actions(granularity=0.2):
    possible = []
    for i in np.linspace(0, 1, int(1 / granularity + 1)).tolist():
        possible.append(round(i,1))    
    actions = []
    for x in possible:
        for y in possible:
            for z in possible:
                if x + y + z == 1:
                    actions.append([x, y, z])
#     print(f"Number of possible actions: {len(actions)}")
    return actions

def init_model(actions, one=60, two=30, activation='sigmoid', seed=0):
    state_size = 6
    n_actions = len(actions)
    
    tensorflow.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    model = Sequential()
    model.add(Embedding(state_size, 20, input_length=1)) # 20
    model.add(Reshape((20,)))
    model.add(Dense(one, activation=activation)) # 80
    model.add(Dense(two, activation=activation)) # 40
    model.add(Dense(n_actions, activation='linear'))
    
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.01))
    return model

def retrain_q_network(q_network, store, batch_size, alpha):
    minibatch = random.sample(store, batch_size)
    for i in minibatch:
        state = i[0]
        action = i[1]
        reward = i[2]
        target = q_network.predict(state)
        
        target[0][action] = target[0][action] + alpha * reward
        
        q_network.fit(np.asarray(state), target, epochs=1, verbose=0)
    return q_network

def choose_action(q_network, state, epsilon, actions, epsilon_greedy=True):   
    max_q = -math.inf
    action = None
    action_indices = list(range(0, len(actions)))
    random.shuffle(action_indices)
    if epsilon_greedy:
        prob = random.uniform(0, 1)
        if prob < epsilon:
            action = random.choice(action_indices)
        else:
            q_values = q_network.predict(state)
#             print(q_values)
            for i, q in enumerate(q_values[0]):
                if q >= max_q:
                    max_q = q
                    action = i
    else:
        q_values = q_network.predict(state)
        for i, q in enumerate(q_values[0]):
            if q >= max_q:
                max_q = q
                action = i
    return action

def predict_individual(date, x, model, column):
    x = x[x['Date'] == date]
    if x.empty:
#         print(date, "empty")
        return 0
    pred = x['pred'].iloc[0]
#     x = x.drop(['Date'], axis=1)
#     x = x.drop([column], axis=1)
#     x = np.asarray(x)
#     x = x.reshape((x.shape[0], 1, x.shape[1]))
#     pred = int(model.predict(x, batch_size=1).round())
    return pred

def predict(date, instruments, data, models):
    state = []
    for symbol in instruments:
        column = symbol + "_relative_change_perc_1"
        ndir = symbol + "_dir"
        nmag = symbol + "_mag"
        
#         model_dir = models[ndir]
        data_dir = data[ndir]
#         model_mag = models[nmag]
        data_mag = data[nmag]
        
        # direction prediction
        pred = predict_individual(date, data_dir, "", column)
        state.append(pred)
        
        # magnitude prediction
        pred = predict_individual(date, data_mag, "", column)
        state.append(pred)
    return state

def translate_action(action_index, actions):
    return actions[action_index]
    

def get_individual_reward(date, x, direction, action, column, capital, risk):
    x = x[x['Date'] == date]
    if x.empty:
        return 0

    relative_change = x[column].iloc[0]
    if np.sign(relative_change) == np.sign(direction):
        relative_change = abs(relative_change)
    else:
        relative_change = -1 * abs(relative_change)
    invested_capital = capital * risk
    profit = relative_change * action * invested_capital
    return profit
    

def get_reward(date, data, state, action, capital, risk):
    profit = 0
    for i, symbol in enumerate(instruments):
        column = symbol + "_relative_change_perc_1"
        ndir = symbol + "_dir"
        data_dir = data[ndir]
        for s in range(len(state)):
            state[s] = int(round(state[s]))
    
        if i == 0:
            pred = state[0]
        elif i == 1:
            pred = state[2]
        elif i == 2:
            pred = state[4]
            
        if pred == 0:
            direction = -1
        else:
            direction = 1
            
        profit += get_individual_reward(date, data_dir, direction, action[i], column, capital, risk)
    return profit

def get_reward_eval(date, data, state, action, capital, risk, metrics):
    profit = 0
    for i, symbol in enumerate(instruments):
        column = symbol + "_relative_change_perc_1"
        ndir = symbol + "_dir"
        data_dir = data[ndir]
        
        for s in range(len(state)):
            state[s] = int(round(state[s]))
    
        if i == 0:
            pred = state[0]
        elif i == 1:
            pred = state[2]
        elif i == 2:
            pred = state[4]
            
        if pred == 0:
            direction = -1
        else:
            direction = 1
            
        individual_reward = get_individual_reward(date, data_dir, direction, action[i], column, capital, risk)
        
        if individual_reward > 0:
            metrics['win_trades'] += 1
            metrics['average_win_trade'] += individual_reward
        elif individual_reward < 0:
            metrics['loss_trades'] += 1
            metrics['average_loss_trade'] += individual_reward
        profit += individual_reward
    return profit, metrics

In [17]:
from tqdm import tqdm

def evaluate_network(q_network, instruments, data, capital, risk, mode):
    
    capitals = []
#     capitals.append(capital)
    data = extract_data(instruments, data, mode)
    dates = data[instruments[0] + "_dir"]['Date'].tolist()
    peak = capital
    valley = capital
    
    metrics = {
        'start_date': dates[0],
        'end_date': dates[-1],
        'profit' : 0,
        'relative_profit': 0,
        'max_drawdown': 0,
        'rel_max_drawdown': 0,
        'max_run_up': 0,
        'rel_max_run_up': 0,
        
        'trades': 0,
        'win_trades': 0,
        'loss_trades': 0,
        'relative_win_trades': 0,
        
        'days_traded': len(dates),
        'win_days': 0,
        'loss_days': 0,
        'relative_win_days': 0,
        
        'average_trade': 0,
        'average_win_trade': 0,
        'average_loss_trade': 0,
        'ratio_wl_trade': 0,
        
        'average_day': 0,
        'average_win_day': 0, 
        'average_loss_day': 0,
        'ratio_wl_day': 0,
        'max_consecutive_wins': 0 
    }
    
    
    actions = init_actions()
    consec_wins = 0
    for d in tqdm(dates, total=len(dates)):
        date = pd.to_datetime(d)
        state = predict(date, instruments, data, models)
        action_index = choose_action(q_network, state, epsilon, actions, False)
        action = translate_action(action_index, actions)
        reward = get_reward(date, data, state, action, capital, risk)
        for a in action:
            if a > 0:
                metrics['trades'] += 1
        reward, metrics = get_reward_eval(date, data, state, action, capital, risk, metrics)

        metrics['profit'] += reward
        if reward >= 0:
            metrics['win_days'] += 1 
            metrics['average_win_day'] += reward
            consec_wins += 1
            if consec_wins > metrics['max_consecutive_wins']:
                metrics['max_consecutive_wins'] = consec_wins
        else:
            consec_wins = 0
            metrics['loss_days'] += 1   
            metrics['average_loss_day'] += reward
            
        capital += reward
        capitals.append(capital)
        
        if capital > peak:
            peak = capital
        else:
            if peak - capital > metrics['max_drawdown']:
                metrics['max_drawdown'] = peak - capital
            if (peak - capital) / peak > metrics['rel_max_drawdown']:
                metrics['rel_max_drawdown'] = (peak - capital) / peak
            
        if capital < valley:
            valley = capital
        else:
            if capital - valley > metrics['max_run_up']:
                metrics['max_run_up'] = capital - valley
            if (capital - valley) / valley > metrics['rel_max_run_up']:
                metrics['rel_max_run_up'] = (capital - valley) / valley
        
    metrics['relative_profit'] = metrics['profit'] / capitals[0]
    metrics['relative_win_trades'] = metrics['win_trades'] / metrics['trades']
    metrics['relative_win_days'] = metrics['win_days'] / metrics['days_traded']
    
    metrics['average_day'] = metrics['average_day'] / len(dates)
    metrics['average_win_day'] = metrics['average_win_day'] / metrics['win_days']
    metrics['average_loss_day'] = abs(metrics['average_loss_day'] / metrics['loss_days'])
    metrics['ratio_wl_day'] = metrics['average_win_day'] / metrics['average_loss_day']
    
    metrics['average_trade'] = metrics['average_trade'] / metrics['trades']
    metrics['average_win_trade'] = metrics['average_win_trade'] / metrics['win_trades']
    metrics['average_loss_trade'] = abs(metrics['average_loss_trade'] / metrics['loss_trades'])
    metrics['ratio_wl_trade'] = metrics['average_win_trade'] / metrics['average_loss_trade']
    
    return metrics, capitals, dates

def run(alpha, gamma, epsilon, decay, episodes, instruments, data, models, capital, risk, mode, one=60, two=30, activation='relu', store_reset=False, batch_size=10, seed=0):
    print(seed)
    actions = init_actions()
    q_network = init_model(actions, one, two, activation, seed)
    random.seed(seed)
    total_rewards = []
    store = []
    
    data = extract_data(instruments, data, mode)
    dates = data[instruments[0] + "_dir"]['Date'].tolist()
    
    for i in range(episodes):
        print(f"Executing episode {i + 1} / {episodes}")
        total_reward = 0
        diffs = []
        
        for d in tqdm(dates, total=len(dates)):
            date = pd.to_datetime(d)
            state = predict(date, instruments, data, models)
            action_index = choose_action(q_network, state, epsilon, actions)
            action = translate_action(action_index, actions)
            reward = get_reward(date, data, state, action, capital, risk)
            store.append([state, action_index, reward])
            
            if len(store) > batch_size:
                q_network = retrain_q_network(q_network, store, batch_size, alpha)
                if store_reset:
                    store = []
            total_reward += reward
            capital += reward

        epsilon = epsilon * decay
        total_rewards.append(total_reward)
    return q_network, total_rewards

In [8]:
def init_models_data():
    instruments = ['SP500', 'NASDAQ', 'US30']

    models = {}
    
    data_dir_sp500 = pd.read_csv("Models/Models Output/SP500_direction_predictions_unrounded.csv", index_col=0)
    data_mag_sp500 = pd.read_csv("Models/Models Output/SP500_magnitude_predictions_unrounded.csv", index_col=0)
    
    data_dir_us30 = pd.read_csv("Models/Models Output/US30_direction_predictions_unrounded.csv", index_col=0)
    data_mag_us30 = pd.read_csv("Models/Models Output/US30_magnitude_predictions_unrounded.csv", index_col=0)
    
    data_dir_nasdaq = pd.read_csv("Models/Models Output/NASDAQ_direction_predictions_unrounded.csv", index_col=0)
    data_mag_nasdaq = pd.read_csv("Models/Models Output/NASDAQ_magnitude_predictions_unrounded.csv", index_col=0)
    
        
#     print(accuracy(data_dir_sp500['pred'], data_dir_sp500['label']))
#     print(accuracy(data_mag_sp500['pred'], data_mag_sp500['label']))
    
#     print(accuracy(data_dir_us30['pred'], data_dir_us30['label']))
#     print(accuracy(data_mag_us30['pred'], data_mag_us30['label']))
    
#     print(accuracy(data_dir_nasdaq['pred'], data_dir_nasdaq['label']))
#     print(accuracy(data_mag_sp500['pred'], data_mag_nasdaq['label']))
    
    data = {
        'SP500_mag': data_mag_sp500,
        'US30_mag': data_mag_us30,
        'NASDAQ_mag': data_mag_nasdaq,
        'SP500_dir': data_dir_sp500,
        'US30_dir': data_dir_us30,
        'NASDAQ_dir': data_dir_nasdaq
    }
    return instruments, data, models

In [15]:
def plot_capitals(dates, capitals):
    fig1 = make_subplots(rows=1, cols=1, specs=[[{'type':'xy'}]])
    
    fig1.add_trace(go.Scatter(x=dates, y=capitals, mode="lines"), row=1, col=1)
#     fig1.add_vline(x=2.5, line_width=3, line_dash="dash", line_color="green")

    fig1.update_layout(
        title = f'Compounded Capital over Time', 
        xaxis1 = dict(title_text = 'Day'),
        yaxis1 = dict(title_text = 'Capital')
    )
    fig1.show()

gamma = 0.1
decay = 1
alpha = 0.2
gamma = 0.1
epsilon = 0.1
decay = 1
episodes = 5
activation = 'relu'
one = 80
two = 40
batch_size = 100

seed = 579
capital = 1000000
risk = 0.05 # capital risked per day


# one = 40
# two = 60
# episodes = 10
# epsilon = 0.1
# alpha = 0.1
# batch_size = 100

one = 40
two = 80
episodes = 20
epsilon = 0.1
alpha = 0.1
batch_size = 100

seed = 579
# print(one, two, episodes, epsilon, alpha)
capital = 1000000
risk = 0.05 # capital risked per day

instruments, data, models = init_models_data()

mode = 'train'
q_network, total_rewards = run(alpha, gamma, epsilon, decay, episodes, instruments, data, models, capital, risk, mode, one, two, activation, True, batch_size, seed)

instruments, data, models = init_models_data()
mode = 'full'
metrics, capitals, dates = evaluate_network(q_network, instruments, data, capital, risk, mode)
# print(f"Profit in evaluation: {profit}")
plot_capitals(dates, capitals)

579


  0%|                                                  | 0/2130 [00:00<?, ?it/s]

Executing episode 1 / 20


  0%|                                          | 4/2130 [00:00<01:09, 30.50it/s]

Executing episode 2 / 20


...

  0%|                                          | 3/2130 [00:00<03:29, 10.15it/s]

Executing episode 19 / 20


  0%|                                          | 3/2130 [00:00<01:11, 29.58it/s]

Executing episode 20 / 20


100%|███████████████████████████████████████| 2633/2633 [01:56<00:00, 22.69it/s]


In [19]:

    
capital = 1000000
risk = 1

print("Evaluating...")

instruments, data, models = init_models_data()
mode = 'train'
metrics, capitals, dates = evaluate_network(q_network, instruments, data, capital, risk, mode)
# print(f"Profit in evaluation: {profit}")
plot_capitals(dates, capitals)
display(metrics)
df = pd.DataFrame({'dates': dates, 'capitals': capitals})
df.to_csv("Portfolio Output/predict_train_dQ_100.csv")


instruments, data, models = init_models_data()
mode = 'val'
metrics, capitals, dates = evaluate_network(q_network, instruments, data, capital, risk, mode)
# print(f"Profit in evaluation: {profit}")
plot_capitals(dates, capitals)
display(metrics)
df = pd.DataFrame({'dates': dates, 'capitals': capitals})
df.to_csv("Portfolio Output/predict_val_dQ_100.csv")

instruments, data, models = init_models_data()
mode = 'test'
metrics, capitals, dates = evaluate_network(q_network, instruments, data, capital, risk, mode)
# print(f"Profit in evaluation: {profit}")
plot_capitals(dates, capitals)
display(metrics)
df = pd.DataFrame({'dates': dates, 'capitals': capitals})
df.to_csv("Portfolio Output/predict_test_dQ_100.csv")

instruments, data, models = init_models_data()
mode = 'full'
metrics, capitals, dates = evaluate_network(q_network, instruments, data, capital, risk, mode)
# print(f"Profit in evaluation: {profit}")
plot_capitals(dates, capitals)
display(metrics)
df = pd.DataFrame({'dates': dates, 'capitals': capitals})
df.to_csv("Portfolio Output/predict_full_dQ_100.csv")

  0%|                                                  | 0/2130 [00:00<?, ?it/s]

Evaluating...


100%|███████████████████████████████████████| 2130/2130 [01:43<00:00, 20.52it/s]


{'start_date': Timestamp('2009-07-17 00:00:00'),
 'end_date': Timestamp('2017-12-29 00:00:00'),
 'profit': 7810330.320801284,
 'relative_profit': 7.782720864145607,
 'max_drawdown': 353196.59817468934,
 'rel_max_drawdown': 0.10474016989012974,
 'max_run_up': 7810330.320801286,
 'rel_max_run_up': 7.810330320801286,
 'trades': 4260,
 'win_trades': 2514,
 'loss_trades': 1746,
 'relative_win_trades': 0.5901408450704225,
 'days_traded': 2130,
 'win_days': 1257,
 'loss_days': 873,
 'relative_win_days': 0.5901408450704225,
 'average_trade': 0.0,
 'average_win_trade': 10338.006776338381,
 'average_loss_trade': 10412.038210145167,
 'ratio_wl_trade': 0.9928898230766526,
 'average_day': 0.0,
 'average_win_day': 16123.619318298031,
 'average_loss_day': 14269.254481442566,
 'ratio_wl_day': 1.1299552712629173,
 'max_consecutive_wins': 11}

100%|█████████████████████████████████████████| 251/251 [00:11<00:00, 21.30it/s]


...

test 2633
test 2640
test 2640
test 2640
test 2640
test 2640


100%|█████████████████████████████████████████| 252/252 [00:11<00:00, 21.42it/s]


{'start_date': Timestamp('2019-01-02 00:00:00'),
 'end_date': Timestamp('2019-12-31 00:00:00'),
 'profit': 141380.05401720275,
 'relative_profit': 0.1389811848241457,
 'max_drawdown': 53704.8179225279,
 'rel_max_drawdown': 0.047818261875412896,
 'max_run_up': 147790.49142681318,
 'rel_max_run_up': 0.14825634458498863,
 'trades': 504,
 'win_trades': 283,
 'loss_trades': 221,
 'relative_win_trades': 0.5615079365079365,
 'days_traded': 252,
 'win_days': 143,
 'loss_days': 109,
 'relative_win_days': 0.5674603174603174,
 'average_trade': 0.0,
 'average_win_trade': 2522.052810630699,
 'average_loss_trade': 2589.8682868383935,
 'ratio_wl_trade': 0.973815086831894,
 'average_day': 0.0,
 'average_win_day': 3993.8317563906044,
 'average_loss_day': 3942.5494233637955,
 'ratio_wl_day': 1.0130074039713761,
 'max_consecutive_wins': 7}

100%|███████████████████████████████████████| 2633/2633 [02:04<00:00, 21.09it/s]


{'start_date': Timestamp('2009-07-17 00:00:00'),
 'end_date': Timestamp('2019-12-31 00:00:00'),
 'profit': 11823128.405835247,
 'relative_profit': 11.781333739304173,
 'max_drawdown': 1360294.825707877,
 'rel_max_drawdown': 0.11079839908405595,
 'max_run_up': 11859846.104365386,
 'rel_max_run_up': 11.859846104365387,
 'trades': 5266,
 'win_trades': 3084,
 'loss_trades': 2182,
 'relative_win_trades': 0.5856437523737182,
 'days_traded': 2633,
 'win_days': 1542,
 'loss_days': 1091,
 'relative_win_days': 0.5856437523737182,
 'average_trade': 0.0,
 'average_win_trade': 14598.480284300922,
 'average_loss_trade': 15214.750133340434,
 'ratio_wl_trade': 0.9594952369484487,
 'average_day': 0.0,
 'average_win_day': 22293.20509430797,
 'average_loss_day': 20671.855040868624,
 'ratio_wl_day': 1.0784327313748046,
 'max_consecutive_wins': 13}

In [45]:
print(seeds_tried)

[624, 164, 631, 266, 71, 709, 326, 493, 157, 399, 762, 224, 125, 788, 219, 534, 12, 935, 886, 688, 183, 579, 394, 895, 288, 757, 104, 22, 964, 977, 208, 20, 455, 562, 740, 823, 237, 708, 568, 465, 662, 524, 273, 220, 652, 218, 957, 930, 418, 970]
